In [2]:
import sqlite3
import pandas as pd
import os

from pathlib import Path

In [3]:
root_dir = Path.cwd().parent.parent.parent

In [4]:
data_dir = root_dir/"All data"/"T20_by_countries"/"Nepal"/"Database"

In [5]:
conn = sqlite3.connect(data_dir/'nepal_cricket.db')
cursor = conn.cursor()


In [6]:
cursor

In [7]:
# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables = [t[0] for t in tables]  # flatten
print(tables)

['male_ball_by_ball', 'sqlite_sequence', 'female_ball_by_ball', 'processing_summary', 'countries', 'teams', 'tournaments', 'umpires', 'players', 'matches']


In [8]:
#show the column heading in matches table and convert date to datetime
cursor.execute("PRAGMA table_info(male_ball_by_ball);")
columns = cursor.fetchall()
column_names = [col[1] for col in columns]
print(column_names)

['id', 'match_id', 'cricinfo_match_id', 'innings_number', 'over_number', 'ball_in_over', 'batsman', 'bowler', 'non_striker', 'fielder', 'runs_batter', 'runs_extras', 'runs_total', 'is_wide', 'is_no_ball', 'is_bye', 'is_leg_bye', 'is_legal_delivery', 'is_wicket', 'dismissal_type', 'wicket_player_out', 'is_caught', 'ball_area', 'shot_area', 'is_drs', 'is_umpires_call', 'milestone_type', 'description']


In [9]:
# first 10 ball by ball data
cursor.execute("SELECT * FROM male_ball_by_ball LIMIT 10;")
ball_by_ball = cursor.fetchall()
for ball in ball_by_ball:
    print(ball)

(1, 691, '1141835', 1, 1, 1, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 0, 0, 0, 0, 0, 0, 0, 1, 0, '', '', 0, '', '', 0, 0, '', '')
(2, 691, '1141835', 1, 1, 2, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 4, 0, 4, 0, 0, 0, 0, 1, 0, '', '', 0, '', '', 0, 0, '', '')
(3, 691, '1141835', 1, 1, 3, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 0, 0, 0, 0, 0, 0, 0, 1, 0, '', '', 0, '', '', 0, 0, '', '')
(4, 691, '1141835', 1, 1, 4, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 2, 0, 2, 0, 0, 0, 0, 1, 0, '', '', 0, '', '', 0, 0, '', '')
(5, 691, '1141835', 1, 1, 5, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 0, 0, 0, 0, 0, 0, 0, 1, 0, '', '', 0, '', '', 0, 0, '', '')
(6, 691, '1141835', 1, 1, 6, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 0, 1, 1, 1, 0, 0, 0, 0, 0, '', '', 0, '', '', 0, 0, '', '')
(7, 691, '1141835', 1, 1, 7, 'TP Visee', 'Sompal Kami', "MP O'Dowd", '', 4, 0, 4, 0, 0, 0, 0, 1, 0, '', '', 0, '', '', 0, 0, '', '')
(8, 691, '1141835', 1, 2, 1, "MP O'Dowd", 'B Regmi', 'TP Visee', '', 

In [6]:
#show the column heading in matches table and convert date to datetime
cursor.execute("PRAGMA table_info(matches);")
columns = cursor.fetchall()
column_names = [col[1] for col in columns]
print(column_names)

cursor.execute("SELECT * FROM matches LIMIT 5;")
matches = cursor.fetchall()
for match in matches:
    print(match)

['id', 'match_id', 'date', 'match_format', 'comments', 'team_1_name', 'team_2_name', 'winner_name', 'host_country_name', 'venue_name', 'venue_id', 'city', 'umpire_1_name', 'umpire_2_name', 'match_referee_name', 'margin', 'margin_type', 'toss_winner_name', 'toss_decision', 'tournament_name', 'season', 'gender', 'player_of_match', 'source_file', 'created_at', 'updated_at']
(1, 691, '2018-07-29', 'T20', 'Marylebone Cricket Club Tri-Nation T20 Series', 'Netherlands', 'Nepal', None, "Lord's", "Lord's", 8026, 'London', 'Rizwan Akram', 'DA Haggo', 'DT Jukes', None, None, 'Nepal', 'field', 'Marylebone Cricket Club Tri-Nation T20 Series', '2018', 'male', '[]', '1141835.json', '2025-11-20 12:56:10', '2025-11-20 12:56:10')
(2, 730, '2019-01-31', 'T20', 'Nepal tour of United Arab Emirates', 'United Arab Emirates', 'Nepal', 'United Arab Emirates', 'ICC Academy', 'ICC Academy', 6569, 'Dubai', 'Akbar Ali', 'Iftikhar Ali', 'Muhammad Javed', 21, 'runs', 'Nepal', 'field', 'Nepal tour of United Arab Emir

In [14]:
# show count by year
query = """
SELECT strftime('%Y', date) AS year, COUNT(*) AS match_count
FROM matches
Where gender='male'
GROUP BY year
ORDER BY year;
"""
df = pd.read_sql_query(query, conn)
df['year'] = pd.to_datetime(df['year'], format='%Y')
print(df)

        year  match_count
0 2014-01-01            3
1 2015-01-01            5
2 2018-01-01            1
3 2019-01-01           19
4 2020-01-01            2
5 2021-01-01            5
6 2022-01-01           17
7 2023-01-01           12
8 2024-01-01           25
9 2025-01-01           17


In [10]:
df_2025 = pd.read_sql("""
SELECT 
    b.*, 
    m.date,
    m.team_1_name, 
    m.team_2_name, 
    m.toss_winner_name, 
    m.toss_decision
FROM male_ball_by_ball b
JOIN matches m 
    ON b.match_id = m.match_id
WHERE strftime('%Y', m.date) = '2025'
""", conn)

In [15]:
# show first 10 rows but for unique cricinfo_match_id only
df_2025.drop_duplicates(subset=['cricinfo_match_id']).head(10)

,id,match_id,cricinfo_match_id,innings_number,over_number,ball_in_over,batsman,bowler,non_striker,fielder,...,shot_area,is_drs,is_umpires_call,milestone_type,description,date,team_1_name,team_2_name,toss_winner_name,toss_decision
0,18363,3131,1479320,1,1,1,A Nagarajan,Karan KC,R Theruvath,,...,,0,0,,,2025-04-09,Qatar,Nepal,Nepal,field
231,18594,3134,1479322,1,1,1,R Sandaruwan,Karan KC,CV Anto,,...,,0,0,,,2025-04-10,Kuwait,Nepal,Nepal,field
469,18832,3137,1479325,1,1,1,Zeeshan Ali,R Dhakal,Anshuman Rath,,...,,0,0,,,2025-04-12,Hong Kong,Nepal,Nepal,field
527,18890,3193,1479327,1,1,1,R Sandaruwan,Karan KC,CV Anto,,...,,0,0,,,2025-04-13,Kuwait,Nepal,Nepal,field
774,19137,3243,1485939,1,1,1,M Levitt,NK Yadav,MP O'Dowd,,...,,0,0,,,2025-06-16,Netherlands,Nepal,Nepal,field
1053,19416,3246,1485940,1,1,1,HG Munsey,DS Airee,MRJ Watt,,...,,0,0,,,2025-06-17,Scotland,Nepal,Nepal,field
1294,19657,3250,1485941,1,1,1,M Levitt,LN Rajbanshi,Vikramjit Singh,,...,,0,0,,,2025-06-19,Netherlands,Nepal,Nepal,field
1539,19902,3253,1485943,1,1,1,HG Munsey,R Dhakal,MRJ Watt,,...,,0,0,,,2025-06-20,Scotland,Nepal,Scotland,bat
1783,20146,3477,1489968,1,1,1,K Bhurtel,AJ Hosein,Aasif Sheikh,,...,,0,0,,,2025-09-27,Nepal,West Indies,West Indies,field
2037,20400,3483,1489969,1,1,1,K Bhurtel,J Blades,Aasif Sheikh,,...,,0,0,,,2025-09-29,Nepal,West Indies,Nepal,bat


In [95]:
df_2025.columns

Index(['id', 'match_id', 'cricinfo_match_id', 'innings_number', 'over_number',
       'ball_in_over', 'batsman', 'bowler', 'non_striker', 'fielder',
       'runs_batter', 'runs_extras', 'runs_total', 'is_wide', 'is_no_ball',
       'is_bye', 'is_leg_bye', 'is_legal_delivery', 'is_wicket',
       'dismissal_type', 'wicket_player_out', 'is_caught', 'ball_area',
       'shot_area', 'is_drs', 'is_umpires_call', 'milestone_type',
       'description', 'date', 'team_1_name', 'team_2_name', 'toss_winner_name',
       'toss_decision'],
      dtype='object')

In [96]:
df_2025.date = pd.to_datetime(df_2025.date)

In [97]:
df_2025['toss_decision'].value_counts()

toss_decision
field    2957
bat       969
Name: count, dtype: int64

In [98]:
# show only value counts by year
df_2025.columns

Index(['id', 'match_id', 'cricinfo_match_id', 'innings_number', 'over_number',
       'ball_in_over', 'batsman', 'bowler', 'non_striker', 'fielder',
       'runs_batter', 'runs_extras', 'runs_total', 'is_wide', 'is_no_ball',
       'is_bye', 'is_leg_bye', 'is_legal_delivery', 'is_wicket',
       'dismissal_type', 'wicket_player_out', 'is_caught', 'ball_area',
       'shot_area', 'is_drs', 'is_umpires_call', 'milestone_type',
       'description', 'date', 'team_1_name', 'team_2_name', 'toss_winner_name',
       'toss_decision'],
      dtype='object')

In [17]:
# If team of interest - in this case Nepal is batting first, in case where they won toss and chose to bat, 
# there already is innings_number =1 , then create a df for team of interest batting
# if team of interest in this case Nepal  is batting second, in case where they won toss and chose to field,
# or opponent won toss and chose to bat, there already is innings number = 2
# the team of intersest batting second case needs to be added to the df for team of interest batting
def is_team_batting(team, row):
    # if toss winner is team and chose to bat and innings number is 1
    # if toss winner is not team and toss_decision is field and innings number is 1
    # if toss winner is not team and chose to bat and innings number is 2
    if (row['toss_winner_name'] == team and row['toss_decision'] == 'bat' and row['innings_number'] == 1):
        return True
    elif (row['toss_winner_name'] != team and row['toss_decision'] == 'field' and row['innings_number'] == 1):
        return True
    elif (row['toss_winner_name'] != team and row['toss_decision'] == 'bat' and row['innings_number'] == 2):
        return True
    else:
        return False




# Apply filter
team_of_interest = 'Nepal'
df_2025_nepal_bat = df_2025[df_2025.apply(lambda row: is_team_batting(team_of_interest, row), axis=1)]
df_2025_others_bat = df_2025[~df_2025.apply(lambda row: is_team_batting(team_of_interest, row), axis=1)]

# Optional: reset index
df_2025_nepal_bat = df_2025_nepal_bat.reset_index(drop=True)
df_2025_others_bat = df_2025_others_bat.reset_index(drop=True)

In [18]:
df_2025_nepal_bat.drop_duplicates(subset=['cricinfo_match_id']).head(10)

,id,match_id,cricinfo_match_id,innings_number,over_number,ball_in_over,batsman,bowler,non_striker,fielder,...,shot_area,is_drs,is_umpires_call,milestone_type,description,date,team_1_name,team_2_name,toss_winner_name,toss_decision
0,20027,3253,1485943,2,1,1,K Bhurtel,BJ McMullen,Aasif Sheikh,,...,,0,0,,,2025-06-20,Scotland,Nepal,Scotland,bat
119,20146,3477,1489968,1,1,1,K Bhurtel,AJ Hosein,Aasif Sheikh,,...,,0,0,,,2025-09-27,Nepal,West Indies,West Indies,field
248,20400,3483,1489969,1,1,1,K Bhurtel,J Blades,Aasif Sheikh,,...,,0,0,,,2025-09-29,Nepal,West Indies,Nepal,bat
376,20634,3488,1489970,1,1,1,K Bhurtel,KR Mayers,Kushal Malla,,...,,0,0,,,2025-09-30,Nepal,West Indies,West Indies,field
500,20835,3507,1503452,1,1,1,K Bhurtel,Aqif Farooq,Aasif Sheikh,,...,,0,0,,,2025-10-08,Nepal,Kuwait,Kuwait,field
621,21307,3516,1503460,1,1,1,K Bhurtel,Junaid Siddique,Aasif Sheikh,,...,,0,0,,,2025-10-12,Nepal,United Arab Emirates,Nepal,bat
743,21552,3519,1503463,1,1,1,K Bhurtel,MM Baig,Aasif Sheikh,,...,,0,0,,,2025-10-13,Nepal,Qatar,Qatar,field
866,21800,3522,1503466,1,1,1,K Bhurtel,Shah Faisal,Aasif Sheikh,,...,,0,0,,,2025-10-15,Nepal,Oman,Nepal,bat
988,22046,3526,1503470,1,1,1,K Bhurtel,DE Burgess,Aasif Sheikh,,...,,0,0,,,2025-10-17,Nepal,Samoa,Samoa,field


In [100]:
df_2025_nepal_bat['cricinfo_match_id'].unique()

array(['1479320', '1479322', '1479327', '1485939', '1485940', '1485941',
       '1485943', '1489968', '1489969', '1489970', '1503452', '1503457',
       '1503460', '1503463', '1503466', '1503470'], dtype=object)

In [101]:
df_2025_others_bat['cricinfo_match_id'].unique()

array(['1479320', '1479322', '1479325', '1479327', '1485939', '1485940',
       '1485941', '1485943', '1489968', '1489969', '1489970', '1503452',
       '1503457', '1503460', '1503463', '1503466', '1503470'],
      dtype=object)

In [102]:
# filters dismissal type is run out
df_2025_others_bat[df_2025_others_bat['dismissal_type'] == 'run out']

,id,match_id,cricinfo_match_id,innings_number,over_number,ball_in_over,batsman,bowler,non_striker,fielder,...,shot_area,is_drs,is_umpires_call,milestone_type,description,date,team_1_name,team_2_name,toss_winner_name,toss_decision
242,18715,3134,1479322,1,20,6,MNM Aslam,NK Yadav,Yasin Patel,"[""Aasif Sheikh"", ""NK Yadav""]",...,,0,0,,,2025-04-10,Kuwait,Nepal,Nepal,field
394,18983,3193,1479327,1,15,5,Bilal Tahir,Karan KC,Muhammad Umar,"[""K Bhurtel"", ""Aasif Sheikh""]",...,,0,0,,,2025-04-13,Kuwait,Nepal,Nepal,field
588,19425,3246,1485940,1,2,4,LR Naylor,Karan KC,BJ McMullen,"[""Lokesh Bam"", ""AK Sah""]",...,,0,0,,,2025-06-17,Scotland,Nepal,Nepal,field
733,19692,3250,1485941,1,6,6,M Levitt,DS Airee,NRJ Croes,"[""RK Paudel""]",...,,0,0,,,2025-06-19,Netherlands,Nepal,Nepal,field
953,20282,3477,1489968,2,2,2,KR Mayers,Karan KC,AA Jangoo,"[""K Bhurtel""]",...,,0,0,,,2025-09-27,Nepal,West Indies,West Indies,field
1022,20351,3477,1489968,2,13,2,KU Carty,K Bhurtel,N Bidaisee,"[""Aasif Sheikh"", ""DS Airee""]",...,,0,0,,,2025-09-27,Nepal,West Indies,West Indies,field
1313,21015,3507,1503452,2,10,3,CV Anto,DS Airee,Yasin Patel,"[""DS Airee""]",...,,0,0,,,2025-10-08,Nepal,Kuwait,Kuwait,field
1389,21091,3512,1503457,1,4,2,K Kadowaki-Fleming,Sompal Kami,Abhishek Anand,"[""DS Airee""]",...,,0,0,,,2025-10-10,Japan,Nepal,Nepal,field
1485,21187,3512,1503457,1,20,1,R Sakurano-Thomas,NK Yadav,B Ito-Davis,"[""Aasif Sheikh"", ""RK Paudel""]",...,,0,0,,,2025-10-10,Japan,Nepal,Nepal,field
1586,21522,3516,1503460,2,16,2,D Parashar,S Lamichhane,R Chopra,"[""DS Airee""]",...,,0,0,,,2025-10-12,Nepal,United Arab Emirates,Nepal,bat


In [103]:
df_2025_nepal_bat['dismissal_type'].value_counts()

dismissal_type
                     1829
caught                 56
bowled                 23
lbw                    14
run out                 9
caught and bowled       6
stumped                 2
retired hurt            1
Name: count, dtype: int64

In [104]:
df_2025_others_bat['dismissal_type'].value_counts()

dismissal_type
                     1858
caught                 67
bowled                 29
run out                16
lbw                     8
stumped                 4
caught and bowled       3
hit wicket              1
Name: count, dtype: int64

In [105]:
# only show matche_id where dismissal_type is run out
df_2025[df_2025['dismissal_type'] == 'run out']['cricinfo_match_id'].unique()

array(['1479322', '1479327', '1485940', '1485941', '1485943', '1489968',
       '1489969', '1489970', '1503452', '1503457', '1503460', '1503463',
       '1503466', '1503470'], dtype=object)

In [106]:
# which players is run out most
df_2025_nepal_bat[df_2025_nepal_bat['dismissal_type'] == 'run out']['batsman'].value_counts()

batsman
Sompal Kami     2
S Jora          2
RK Paudel       1
Rupesh Singh    1
Aasif Sheikh    1
K Bhurtel       1
NK Yadav        1
Name: count, dtype: int64

In [107]:
# is there a fielder involved in most run outs, also show the crcinfo id of those matches

(
    df_2025_others_bat[df_2025_others_bat['dismissal_type'] == 'run out']
    .groupby('fielder')
    .agg(
        runouts=('fielder', 'count'),
        match_ids=('cricinfo_match_id', lambda x: sorted(set(x)))
    )
    .sort_values('runouts', ascending=False)
)

,runouts,match_ids
fielder,,
"[""DS Airee""]",5,"[1503452, 1503457, 1503460, 1503463, 1503466]"
"[""Aasif Sheikh"", ""RK Paudel""]",2,"[1503457, 1503460]"
"[""Aasif Sheikh"", ""DS Airee""]",1,[1489968]
"[""Aasif Sheikh"", ""NK Yadav""]",1,[1479322]
"[""Aasif Sheikh"", ""S Jora""]",1,[1503460]
"[""K Bhurtel"", ""Aasif Sheikh""]",1,[1479327]
"[""K Bhurtel""]",1,[1489968]
"[""Karan KC""]",1,[1503470]
"[""Lokesh Bam"", ""AK Sah""]",1,[1485940]


In [108]:
df_2025_others_bat[df_2025_others_bat['dismissal_type'] == 'run out']['batsman'].value_counts()

batsman
MNM Aslam             1
Bilal Tahir           1
LR Naylor             1
M Levitt              1
KR Mayers             1
KU Carty              1
CV Anto               1
K Kadowaki-Fleming    1
R Sakurano-Thomas     1
D Parashar            1
Haider Ali            1
Junaid Siddique       1
MM Baig               1
Zubair Ali            1
Aamir Kaleem          1
F Suluoto             1
Name: count, dtype: int64